# Notebook Demo
Based on [tensorflow/datasets:docs/keras_example](https://github.com/tensorflow/datasets/blob/master/docs/keras_example.ipynb)

In [ ]:
# Basic dependencies
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets
from tqdm.keras import TqdmCallback

In [ ]:
# Download training & test data
(ds_train, ds_test), ds_info = tensorflow_datasets.load(
    'mnist', split=['train', 'test'], shuffle_files=True, as_supervised=True, with_info=True)

In [ ]:
# Preprocess data
def norm_img(image, label):
    """Convert images `uint8` -> `float32`"""
    return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(norm_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(norm_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [ ]:
# Create model
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [ ]:
# Train model
model.fit(ds_train, epochs=6, validation_data=ds_test, verbose=0, callbacks=[TqdmCallback()]);

In [ ]:
# Show some test results
test_imgs, test_labs = next(iter(ds_test))
_, axs = plt.subplots(1, 4, figsize=(9, 3))
for i in range(4):
    plt.sca(axs[i])
    plt.imshow(test_imgs[i])
    plt.xticks([])
    plt.yticks([])
    plt.title(f"{test_labs[i]} -> {model.predict(test_imgs[i:i+1]).argmax()}")
plt.tight_layout()